<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Clockify - Create time entries database on a workspace

**Tags:** #clockify #timeentry #database #workspace #user #create

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-07-26 (Created: 2023-07-26)

**Description:** This notebook creates a time entries database on a specific timeframe, adding client, project and task name. It is usefull for organizations to track time entries and optimize their workflow.

**References:**
- [Clockify API Documentation](https://docs.clockify.me/#tag/Time-entry/operation/getTimeEntries)

## Input

### Import libraries

In [1]:
import requests
import naas
import pandas as pd
from datetime import datetime

### Setup Variables
- `api_key`: [Get your API key](https://clockify.me/user/settings)
- `workspace_id`: ID of the workspace
- `user_id`: ID of the user to get time entries from
- `start_date`: Start date of the timeframe to create the time entries database
- `end_date`: End date of the timeframe to create the time entries database

In [5]:
api_key = naas.secret.get("CLOCKIFY_API_KEY") or "YOUR_API_KEY"
workspace_id = "626f9e3b36c2670314c0386e" #"<WORKSPACE_ID>"
start_date = "2023-01-01"
end_date = datetime.now().astimezone().isoformat()#.strftime("%Y-%m-%d")

## Model

### Function: Flatten the nested dict

In [6]:
# Flatten the nested dict
def flatten_dict(d, parent_key='', sep='_'):
    """
    Flattens a nested dictionary into a single level dictionary.

    Args:
        d (dict): A nested dictionary.
        parent_key (str): Optional string to prefix the keys with.
        sep (str): Optional separator to use between parent_key and child_key.

    Returns:
        dict: A flattened dictionary.
    """
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

### Function: Get referentials from workspace

In [26]:
def get_data(api_key, workspace_id, endpoint):
    # Init
    page = 1
    df = pd.DataFrame()
    
    while True:
        # Requests
        url = f"https://api.clockify.me/api/v1/workspaces/{workspace_id}/{endpoint}"
        headers = {
            "X-Api-Key": api_key
        }
        params = {
            "page": page,
            "page-size": 100
        }
        res = requests.get(url, headers=headers, params=params)
        data = res.json()
        if len(data) > 0:
            for d in data:
                res = flatten_dict(d)
                tmp_df = pd.DataFrame([res])
                df = pd.concat([df, tmp_df])
        else:
            break
        page += 1
    return df.reset_index(drop=True)

### Get all users

In [29]:
df_users = get_data(api_key, workspace_id, "users")
df_users = df_users.rename(columns={"id": "userId", "name": "userName"})
df_users = df_users[["userId", "userName", "email"]]
print("Users fetched:", len(df_users))
df_users.head(1)

Users fetched: 9


,userId,userName,email
0,643d11b8d00ede21e7bb0732,Bijo Babu,bijo.babu@bridge-global.com


### Get time entries

In [33]:
def get_time_entries(
    api_key,
    workspace_id,
    user_id,
    start_date,
    end_date
):
    # Init
    start_date = datetime.strptime(start_date, "%Y-%m-%d").astimezone().isoformat() # Format date
    page = 1
    df = pd.DataFrame()
    
    # Get raw data
    while True:
        url = f"https://api.clockify.me/api/v1/workspaces/{workspace_id}/user/{user_id}/time-entries"
        headers = {"X-Api-Key": api_key}
        params = {
            "start": start_date,
            "end": end_date,
            "page": page,
            "page-size": 100
        }
        res = requests.get(url, headers=headers, params=params)
        data = res.json()
        if len(data) > 0:
            for d in data:
                res = flatten_dict(d)
                tmp_df = pd.DataFrame([res])
                df = pd.concat([df, tmp_df]).reset_index(drop=True)
        else:
            break
        page += 1
    return df.reset_index(drop=True)

# Init
database = pd.DataFrame()

# Loop
for row in df_users.itertuples():
    user_id = row.userId
    user_name = row.userName
    user_email = row.email
    
    # Get entries
    df_time_entries = get_time_entries(api_key, workspace_id, user_id, start_date, end_date)

    # Concat
    database = pd.concat([database, df_time_entries]).reset_index(drop=True)
    
print("Time entries fetched:", len(database))
database.head(3)

Time entries fetched: 1446


,id,description,tagIds,userId,billable,taskId,projectId,timeInterval_start,timeInterval_end,timeInterval_duration,workspaceId,isLocked,customFieldValues,type,kioskId
0,64c1e8bfc357e9197ef110d8,#428 Bugs using plugin: logo open ai not displ...,None,643d11b8d00ede21e7bb0732,False,None,636a73439ddd1626891e7e84,2023-07-27T03:47:09Z,2023-07-27T08:35:50Z,PT4H48M41S,626f9e3b36c2670314c0386e,False,[],REGULAR,None
1,64c0fc6e9479511b7231e331,#430 Hide regenerate response when no messages,None,643d11b8d00ede21e7bb0732,False,None,636a73439ddd1626891e7e84,2023-07-26T10:58:54Z,2023-07-26T12:21:03Z,PT1H22M9S,626f9e3b36c2670314c0386e,False,[],REGULAR,None
2,64c093fbc357e9197ee947d1,#426 feat: Improve how model are added and act...,None,643d11b8d00ede21e7bb0732,False,None,636a73439ddd1626891e7e84,2023-07-26T10:00:00Z,2023-07-26T10:30:00Z,PT30M,626f9e3b36c2670314c0386e,False,[],REGULAR,None


### Get all projects

In [30]:
df_projects = get_data(api_key, workspace_id, "projects")
df_projects = df_projects.rename(columns={"id": "projectId", "name": "projectName"})
df_projects = df_projects[["projectId", "projectName", "clientId"]]
print("Projects fetched:", len(df_projects))
df_projects.head(1)

Projects fetched: 34


,projectId,projectName,clientId
0,62ea33a97edb1761ccccbc11,Admin,626fef6d1541a9131641de2d


### Get all clients

In [31]:
df_clients = get_data(api_key, workspace_id, "clients")
df_clients = df_clients.rename(columns={"id": "clientId", "name": "clientName"})
df_clients = df_clients[["clientId", "clientName"]]
print("Clients fetched:", len(df_clients))
df_clients.head(1)

Clients fetched: 12


,clientId,clientName
0,626fefff1541a9131641e5fc,Arista


## Output

### Create final database
Enrich data with referentials from workspace

In [38]:
final_db = database.copy()

# Final DB
final_db = pd.merge(final_db, df_users, how="left", on="userId")
final_db = pd.merge(final_db, df_projects, how="left", on="projectId")
final_db = pd.merge(final_db, df_clients, how="left", on="clientId")

# Select column
to_select = [
    "id",
    "description",
    "isLocked",
    "timeInterval_start",
    "timeInterval_end",
    "timeInterval_duration",
    "userId",
    "userName",
    "email",
    "projectId",
    "projectName",
    "clientId",
    "clientName",
]
final_db = final_db[to_select]
print("Time entries fetched:", len(final_db))
final_db.head(1)

Time entries fetched: 1446


,id,description,isLocked,timeInterval_start,timeInterval_end,timeInterval_duration,userId,userName,email,projectId,projectName,clientId,clientName
0,64c1e8bfc357e9197ef110d8,#428 Bugs using plugin: logo open ai not displ...,False,2023-07-27T03:47:09Z,2023-07-27T08:35:50Z,PT4H48M41S,643d11b8d00ede21e7bb0732,Bijo Babu,bijo.babu@bridge-global.com,636a73439ddd1626891e7e84,Platform,626fef6d1541a9131641de2d,Naas
